# Ironman Test

Ironman can perform joint fits of the photometry, out-of-transit radial velocities, and Rossiter-McLaughlin effect for a given exoplanet system.

First, we need to import our data properly

In [1]:
import ironman1
import pandas as pd
import numpy as np

We can create a function to read our different files

In [2]:
def read_file(archivo):
    file = open(archivo,"r")
    bjd = []
    rv = []
    e_rv = []
    for linea in file:
        if linea[0]!="#":
            linea = linea.split()
            bjd.append(float(linea[0]))
            rv.append(float(linea[1]))
            e_rv.append(float(linea[2]))
            #print(linea)
    bjd = np.array(bjd)
    rv = np.array(rv)
    e_rv = np.array(e_rv)
    return bjd,rv,e_rv

Then, we can use the function to read the radial velocities (RVs) of different instruments. All RVs must be in m/s

In [3]:
bjd_chiron, rv_chiron, e_rv_chiron = read_file("Example_Data/chiron_rvs.txt")

bjd_mine, rv_mine, e_rv_mine = read_file("Example_Data/minerva_rvs.txt")

bjd_feros, rv_feros, e_rv_feros = read_file("Example_Data/feros_rvs.txt")
rv_feros = rv_feros*1000.0
e_rv_feros = e_rv_feros*1000.0

Next, we need to import our photometric data

In [4]:
tess1 = pd.read_csv("Example_Data/TESS_LC_detrended_Y1.csv")
bjd_tess1 = tess1.time.values
flux_tess1 = tess1.flux.values
e_flux_tess1 = tess1.flux_err.values

tess2 = pd.read_csv("Example_Data/TESS_LC_detrended_Y2.csv")
bjd_tess2 = tess2.time.values
flux_tess2 = tess2.flux.values
e_flux_tess2 = tess2.flux_err.values

tess3 = pd.read_csv("Example_Data/TESS_LC_detrended_Y3.csv")
bjd_tess3 = tess3.time.values
flux_tess3 = tess3.flux.values
e_flux_tess3 = tess3.flux_err.values

FITO = pd.read_csv("Example_Data/FITO_det.csv")
bjd_fito = FITO.time.values
flux_fito = FITO.flux.values
flux_err_fito = FITO.flux_err.values

B = pd.read_csv("Example_Data/LCOGT_B_det.csv")
bjd_LCOGT_B = B.time.values
flux_LCOGT_B = B.flux.values
flux_err_LCOGT_B = B.flux_err.values

zs = pd.read_csv("Example_Data/LCOGT_zs_det.csv")
bjd_LCOGT_zs = zs.time.values
flux_LCOGT_zs = zs.flux.values
flux_err_LCOGT_zs = zs.flux_err.values

A1 = pd.read_csv("Example_Data/ASTEP1_det.csv")
bjd_astep1 = A1.time.values
flux_astep1 = A1.flux.values
flux_err_astep1 = A1.flux_err.values

A2 = pd.read_csv("Example_Data/ASTEP2_det.csv")
bjd_astep2 = A2.time.values
flux_astep2 = A2.flux.values
flux_err_astep2 = A2.flux_err.values

And finally the RM data

In [5]:
bjd_RM, rv_RM, e_rv_RM = read_file("Example_Data/TOI3362_RVs.ascii")
rv_RM = rv_RM*1000.0
e_rv_RM = e_rv_RM*1000.0

We need to create 9 dictionaries

In [6]:
times_lc, fluxes, fluxes_error = {}, {}, {}
times_rvs, rvs, rvs_err =  {}, {}, {}
times_RM, RM, RM_err =  {}, {}, {}

And then add our data to the respective dictionary

In [9]:
times_rvs["CHIRON"], rvs["CHIRON"], rvs_err["CHIRON"] = bjd_chiron, rv_chiron, e_rv_chiron
times_rvs["FEROS"], rvs["FEROS"], rvs_err["FEROS"] = bjd_feros, rv_feros, e_rv_feros
times_rvs["MINE"], rvs["MINE"], rvs_err["MINE"] = bjd_mine, rv_mine, e_rv_mine

times_lc["FITO"], fluxes["FITO"], fluxes_error["FITO"] = bjd_fito, flux_fito, flux_err_fito
times_lc["TESSY1"], fluxes["TESSY1"], fluxes_error["TESSY1"] = bjd_tess1, flux_tess1, e_flux_tess1
times_lc["TESSY2"], fluxes["TESSY2"], fluxes_error["TESSY2"] = bjd_tess2, flux_tess2, e_flux_tess2
times_lc["TESSY3"], fluxes["TESSY3"], fluxes_error["TESSY3"] = bjd_tess3, flux_tess3, e_flux_tess3
times_lc["B"], fluxes["B"], fluxes_error["B"] = bjd_LCOGT_B, flux_LCOGT_B, flux_err_LCOGT_B
times_lc["ZS"], fluxes["ZS"], fluxes_error["ZS"] = bjd_LCOGT_zs, flux_LCOGT_zs, flux_err_LCOGT_zs
times_lc["ASTEP"], fluxes["ASTEP"], fluxes_error["ASTEP"] = np.concatenate((bjd_astep1,bjd_astep2)), np.concatenate((flux_astep1,flux_astep2)), np.concatenate((flux_err_astep1,flux_err_astep2))

times_RM["ESPRESSO"], RM["ESPRESSO"], RM_err["ESPRESSO"] = bjd_RM, rv_RM, e_rv_RM

For the photometric and RM data, we need to create an extra dictionary containing the exposure times

In [8]:
exp = {"FITO": False, "ESPRESSO": 310./60./60./24., "TESSY1": 30./60./24., "TESSY2": 10./60./24., "TESSY3": False, "B": False, "ZS": False, "ASTEP": False}

Then, we have to add those dictionaries to the Ironman data organizer

In [10]:
my_data = ironman1.Data_Org(lc_time=times_lc,lc_flux=fluxes,lc_flux_err=fluxes_error,rv_time=times_rvs,rv=rvs,rv_err=rvs_err,rm_time=times_RM,rm=RM,rm_err=RM_err,verbose = True,exp_times=exp)

Reading LCs...
Reading RVs...
Reading RM data...


Also, we need to add our priors. This function needs a Data_Org

In [11]:
my_priors = ironman1.Priors("priors_example.dat",my_data)

Priors dictionary ready...
Detecting fixed parameters...


Finally, using these two previous objects, we create our Fit object

In [12]:
my_fit = ironman1.Fit(my_data,my_priors)

To run it:

In [ ]:
post = my_fit.run(n_live=3500, nthreads = 40)

The variable post contains the flatchain. Using 3500 n_live and 40 threads it takes ~ 1 hour  to run

We can save the flatchain using

In [ ]:
np.savetxt('flatchain.csv', post, delimiter=',')

We can analyse it later with another script